In [1]:
import paho.mqtt.client as mqtt
import ssl
import json
import re
from datetime import datetime

MQTT_BROKER = "fc8731aef5304f3aad37fa03be01e517.s1.eu.hivemq.cloud"
MQTT_PORT = 8883
MQTT_USERNAME = "finalProject"
MQTT_PASSWORD = "Fb123456"

TOPICS = {
    "heart_rate": "device/sensor/heart_rate",
    "oxygen_level": "device/sensor/spo2",
    "temperature": "device/sensor/temperature"
}

OUTPUT_FILE = "sensor_readings.txt"

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ Connected to MQTT broker")
        for topic in TOPICS.values():
            client.subscribe(topic)
            print(f"📡 Subscribed to {topic}")
    else:
        print(f"❌ Failed to connect, return code: {rc}")

def on_message(client, userdata, msg):
    try:
        raw_payload = msg.payload.decode()
        print(f"📥 Raw message on {msg.topic}: {raw_payload}")
        try:
            payload = json.loads(raw_payload)
            value = float(payload["value"])
        except:
            match = re.search(r"[-+]?\d*\.\d+|\d+", raw_payload)
            if match:
                value = float(match.group())
            else:
                raise ValueError("No numeric value found in message")

        topic_name = [k for k, v in TOPICS.items() if v == msg.topic][0]
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        log_line = f"{timestamp} | {topic_name}: {value}"
        print(log_line)

        with open(OUTPUT_FILE, "a") as f:
            f.write(log_line + "\n")

    except Exception as e:
        print(f"⚠️ Error processing message: {e}")

client = mqtt.Client()
client.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)

client.tls_set()

client.on_connect = on_connect
client.on_message = on_message

try:
    print("⏳ Connecting to broker...")
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_forever()
except Exception as e:
    print(f"❌ Connection error: {e}")


C:\Users\clt\AppData\Local\Temp\ipykernel_1580\498831490.py:55: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


⏳ Connecting to broker...
✅ Connected to MQTT broker
📡 Subscribed to device/sensor/heart_rate
📡 Subscribed to device/sensor/spo2
📡 Subscribed to device/sensor/temperature
📥 Raw message on device/sensor/heart_rate: Heart rate: 85.3 bpm
2025-06-21 20:13:48 | heart_rate: 85.3
📥 Raw message on device/sensor/temperature: Temperature: 34.81 *C
2025-06-21 20:13:48 | temperature: 34.81
📥 Raw message on device/sensor/spo2: SpO2: 97.0%
2025-06-21 20:13:48 | oxygen_level: 2.0
📥 Raw message on device/sensor/heart_rate: Heart rate: 83.9 bpm
2025-06-21 20:13:51 | heart_rate: 83.9
📥 Raw message on device/sensor/temperature: Temperature: 34.91 *C
2025-06-21 20:13:51 | temperature: 34.91
📥 Raw message on device/sensor/spo2: SpO2: 97.0%
2025-06-21 20:13:51 | oxygen_level: 2.0
📥 Raw message on device/sensor/temperature: Temperature: 34.99 *C
2025-06-21 20:13:54 | temperature: 34.99
📥 Raw message on device/sensor/heart_rate: Heart rate: 75.3 bpm
2025-06-21 20:13:54 | heart_rate: 75.3
📥 Raw message on devi

KeyboardInterrupt: 